In [16]:
import pandas as pd
import os

file = input("File name: ")
data = pd.read_excel(file)
cont = True

In [17]:
def output(df):
    path = 'user_output'
    if not os.path.exists(path):
        os.makedirs(path)
    if not df.empty:
        file_name = input("file name: ")
        df.to_excel(path + "/" + file_name)
    return

In [18]:
def family(df):
    t = input('Type: ')
    if (t=='PDC'):
        df = pd.DataFrame(df.query('`Bit Family` == "FCS" or `Bit Family` == "FCM"'))
    elif (t=='TCR'):
        df = pd.DataFrame(df.query('`Bit Family` == "RCT" or `Bit Family` == "RCI"'))
    else:
        df = pd.DataFrame(df.query('`Bit Family` == @t'))
    return df

In [19]:
def size(df): 
    s = input("Size? ")
    try:
        s.split()
        s = s.split()
        s = [float(s[0]), float(s[1])]
        df = df[df['Bit Size (in)'].isin(s)]
    except IndexError:
        s = float(s[0])
        df = pd.DataFrame(df.query('`Bit Size (in)` == @s'))
    #df = df[df['Bit Size (in)'].isin(s)]
    #df = pd.DataFrame(df.query('`Bit Size (in)` == @s'))
    return df

In [20]:
def country(df):
    df['Country'] = df['Country'].str.lower()
    c = input("Which country or countries?")
    c = c.split()
    df = df[df['Country'].str.contains('|'.join(c), na=False)]
    #df = pd.DataFrame(df.query('Country == @c'))
    return df

In [21]:
def company(df):
    df['Operator'] = df['Operator'].str.lower()
    c = input("Which company or companies?")
    c = c.split()
    df = df[df['Operator'].str.contains('|'.join(c), na=False)]
    #df = pd.DataFrame(df.query('Operator == @c'))
    return df

In [22]:
def depth(df):
    d1 = float(input("Lower limit?"))
    df = pd.DataFrame(df.query('`MD In (m)` >= @d1'))
    d2 = float(input("Upper limit?"))
    if(d2!=''):
        df = pd.DataFrame(df.query('`MD In (m)` <= @d2'))
    return df

In [23]:
def date(df):
    df['Run Date'] = pd.to_datetime(df['Run Date'], utc=False)
    s = input("Start Date:")
    s = pd.to_datetime(s, utc=False)
    df = df[df['Run Date'] >= s]
    e = input("End Date:")
    if(e!=''):
        e = pd.to_datetime(e, utc=False)
        df = df[df['Run Date'] <= e]
    return df

In [24]:
def best_five(col, df):
    df = df.sort_values(by=col, ascending=False)
    df = df.head(5)
    return df

In [25]:
def meterage(df):
    name = [col for col in df.columns if 'Run Length' in col]
    try:
        name = name[0]
        index = df.columns.get_loc(name)
        #df['Serial Number'] = df['Serial Number'].fillna('blank')
        df['cumm Meterage'] = df.groupby('Serial Number')[name].transform('cumsum')
        column = df.pop('cumm Meterage')
        df.insert(index+1, 'Cumm Meterage', column)
    except KeyError:
        print('"Run Length" column is missing or present with a different name')
    return df

In [26]:
def average(col, df):
    df['Average ROP (m/hr)'] = df[col].mean()
    return df

In [27]:
def rop(df):
    r = input("Value")
    col = 'ROP (m/hr)'
    df = df[df['ROP (m/hr)']>=r]
    print("Do you want to perform any other action on this column?")
    a = input("1. best_five \n 2. average \n 3. both")
    if(a=='1'):
        df = best_five(col, df)
    elif(a=='2'):
        df = average(col, df)
    elif(a=='3'):
        df = best_five(col, df)
        df = average(col, df)
    return df

In [28]:
def custom(df):
    col = input("Which column do you wanna filter?")
    val = input("Value to be filtered?")
    try:
        val.split()
        s, e = val.split()
    except ValueError:
        s = val 
    try:
        float(s)
        num = True
    except ValueError:
        num = False
    if(num==True):
        s = float(s)
        e = float(e)
        df = pd.DataFrame(df.query('`{0}` >= @s'.format(col)))
        df = pd.DataFrame(df.query('`{0}` <= @e'.format(col)))
    else:
        df = pd.DataFrame(df.query('`{0}` == @val'.format(col)))
    return df

In [29]:
def filters(df):
    print("Which filter do you wanna use?")
    print("1. Size \n2. Type \n3. Country \n4. Company \n5. Depth \n6. Date \n7. Meterage \n8. ROP \n9. Custom")
    f = input("1. Size 2. Type 3. Country 4. Company 5.Depth 6. Date 7. Meterage 8. ROP 9. Custom")
    if(f=='1'):
        df = size(df)
    elif(f=='2'):
        df = family(df)
    elif(f=='3'):
        df = country(df)
    elif(f=='4'):
        df = company(df)
    elif(f=='5'):
        df = depth(df)
    elif(f=='6'):
        df = date(df)
    elif(f=='7'):
        df = meterage(df)
    elif(f=='8'):
        df = rop(df)
    elif(f=='9'):
        df = custom(df)
    return df    

In [30]:
while(cont==True):    
    print("What do you wanna do?")
    print("1. Output \n2. Filter")
    a = input("1. Output \n 2. Filter")
    if(a=='1'):
        output(data)
        cont=False
    elif(a=='2'):
        data=filters(data)

What do you wanna do?
1. Output 
2. Filter
Which filter do you wanna use?
1. Size 
2. Type 
3. Country 
4. Company 
5. Depth 
6. Date 
7. Meterage 
8. ROP 
9. Custom
What do you wanna do?
1. Output 
2. Filter
